# PART C

In [17]:
import pandas as pd
import numpy as np
from collections import Counter
from math import log

# Función para cargar el dataset
def cargar_dataset(ruta_csv, nrows=100000): #nrows=numfilas
    # Saltar la primera fila usando skiprows=1
    df = pd.read_csv(ruta_csv, sep=';', skiprows=1, header=None, names=["id", "text", "date", "label"], low_memory=False, nrows=nrows) #,nrows=nrows
    
    # Convertir la columna 'label' a tipo int
    df["label"] = pd.to_numeric(df["label"], errors='coerce')  # Coerce convierte valores no numéricos a NaN
    
    # Eliminar filas no válidas si las hay
    df = df.dropna(subset=["text", "label"]).reset_index(drop=True)
    df["label"] = df["label"].astype(int)  # Convertir la columna 'label' a entero
    return df



# División balanceada en train y test
def dividir_train_test(df, test_size=0.2, seed=42):
    # Estratificación por etiqueta
    from sklearn.model_selection import train_test_split
    train_df, test_df = train_test_split(df, test_size=test_size, random_state=seed, stratify=df['label'])
    return train_df, test_df

# Construir diccionario y estadísticas
def construir_diccionario(train_df, min_freq=1):
    pos_texts = train_df[train_df['label'] == 1]['text']
    neg_texts = train_df[train_df['label'] == 0]['text']

    word_counts_pos = Counter()
    word_counts_neg = Counter()

    for text in pos_texts:
        word_counts_pos.update(text.split())

    for text in neg_texts:
        word_counts_neg.update(text.split())

    # Reducir diccionario eliminando palabras con frecuencia menor a min_freq
    if min_freq > 1:
        word_counts_pos = Counter({k: v for k, v in word_counts_pos.items() if v >= min_freq})
        word_counts_neg = Counter({k: v for k, v in word_counts_neg.items() if v >= min_freq})

    total_tweets = len(train_df)
    p_pos = len(pos_texts) / total_tweets
    p_neg = len(neg_texts) / total_tweets

    return {
        'word_counts_pos': word_counts_pos,
        'word_counts_neg': word_counts_neg,
        'p_pos': p_pos,
        'p_neg': p_neg,
        'vocab_size': len(set(word_counts_pos.keys()).union(set(word_counts_neg.keys())))
    }


# Clasificación de un tweet usando Naïve Bayes
def predecir_tweet_optimizado(tweet, stats, alpha=1):
    log_prob_pos = log(stats['p_pos'])
    log_prob_neg = log(stats['p_neg'])
    vocab_size = stats['vocab_size']

    # Convertir los contadores en diccionarios para acceso rápido
    words_pos = stats['word_counts_pos']
    words_neg = stats['word_counts_neg']

    # Precomputar totales
    total_words_pos = sum(words_pos.values())
    total_words_neg = sum(words_neg.values())

    # Tokenización
    words = tweet.split()

    # Convertir en arrays para cálculos rápidos
    word_probs_pos = np.array([log((words_pos.get(word, 0) + alpha) / (total_words_pos + alpha * vocab_size)) for word in words])
    word_probs_neg = np.array([log((words_neg.get(word, 0) + alpha) / (total_words_neg + alpha * vocab_size)) for word in words])

    # Sumar probabilidades en logaritmo
    log_prob_pos += word_probs_pos.sum()
    log_prob_neg += word_probs_neg.sum()

    return 1 if log_prob_pos > log_prob_neg else 0

# Evaluación del modelo
def evaluar_modelo_optimizado(test_df, stats):
    # Aplicar la predicción a cada tweet
    y_pred = test_df['text'].apply(lambda tweet: predecir_tweet_optimizado(tweet, stats))
    y_true = test_df['label'].values

    # Calcular la accuracy
    accuracy = np.mean(y_pred == y_true)
    print("accuracy del modelo:", accuracy)
    return accuracy

# Main
if __name__ == "__main__":
    ruta_csv = "./CRI_Practica3/FinalStemmedSentimentAnalysisDataset.csv"

    # Cargar datos
    df = cargar_dataset(ruta_csv)
    print("Ejemplo de datos:\n", df.head())

    # División en train y test
    train_df, test_df = dividir_train_test(df)
    print(f"\nTamaño del conjunto de entrenamiento: {len(train_df)}")
    print(f"Tamaño del conjunto de prueba: {len(test_df)}")

    # Construir diccionario y estadísticas
    stats = construir_diccionario(train_df)

    # Evaluar el modelo
    evaluar_modelo_optimizado(test_df, stats)


Ejemplo de datos:
    id                                               text        date  label
0   1                       is so sad for my apl friend   04/03/2015      0
1   2                         i miss the new moon trail   06/10/2015      0
2   3                              omg it already 730 o   03/04/2015      1
3   4   omgag im sooo im gunn cry i've been at thi de...  13/11/2015      0
4   5                     i think mi bf is che on me tt   10/08/2015      0

Tamaño del conjunto de entrenamiento: 79992
Tamaño del conjunto de prueba: 19998
accuracy del modelo: 0.7596759675967597


# PART B

In [16]:
import pandas as pd
import numpy as np
from collections import Counter
from math import log

# Clasificación de un tweet usando Naïve Bayes
def predecir_tweet_optimizado(tweet, stats, alpha=1):
    log_prob_pos = log(stats['p_pos'])
    log_prob_neg = log(stats['p_neg'])
    vocab_size = stats['vocab_size']

    # Convertir los contadores en diccionarios para acceso rápido
    words_pos = stats['word_counts_pos']
    words_neg = stats['word_counts_neg']

    # Precomputar totales
    total_words_pos = sum(words_pos.values())
    total_words_neg = sum(words_neg.values())

    # Tokenización
    words = tweet.split()

    # Convertir en arrays para cálculos rápidos
    word_probs_pos = np.array([log((words_pos.get(word, 0) + alpha) / (total_words_pos + alpha * vocab_size)) for word in words])
    word_probs_neg = np.array([log((words_neg.get(word, 0) + alpha) / (total_words_neg + alpha * vocab_size)) for word in words])

    # Sumar probabilidades en logaritmo
    log_prob_pos += word_probs_pos.sum()
    log_prob_neg += word_probs_neg.sum()

    return 1 if log_prob_pos > log_prob_neg else 0

def evaluar_modelo(test_df, stats):
    y_true = test_df['label'].values
    y_pred = [predecir_tweet_optimizado(row['text'], stats) for _, row in test_df.iterrows()]
    return np.mean(y_pred == y_true)

# Estrategia 1: Aumentar el tamaño del conjunto de entrenamiento
def estrategia_1(df, incrementos):
    print("\nEstrategia 1: Aumentar tamaño de entrenamiento (diccionario variable)")
    resultados = []
    for n in incrementos:
        train_df, test_df = dividir_train_test(df[:n])
        stats = construir_diccionario(train_df)
        acc = evaluar_modelo(test_df, stats)
        resultados.append((n, acc))
        print(f"Tamaño Train: {n}, accuracy: {acc:.4f}")
    return resultados

# Estrategia 2: Reducir el diccionario
def estrategia_2(df, min_freqs):
    print("\nEstrategia 2: Modificar tamaño del diccionario")
    train_df, test_df = dividir_train_test(df)
    resultados = []
    for min_freq in min_freqs:
        stats = construir_diccionario(train_df, min_freq=min_freq)
        acc = evaluar_modelo(test_df, stats)
        resultados.append((min_freq, acc))
        print(f"Min. Frecuencia: {min_freq}, accuracy: {acc:.4f}")
    return resultados

# Estrategia 3: Mantener el tamaño del diccionario fijo y variar el tamaño del conjunto de entrenamiento
def estrategia_3(df, train_sizes):
    print("\nEstrategia 3: Modificar tamaño del conjunto de entrenamiento (diccionario fijo)")
    # Dividimos una sola vez el dataset para obtener el conjunto de test fijo
    train_df, test_df = dividir_train_test(df, test_size=0.2)    
    # Construir el diccionario usando el conjunto completo de entrenamiento
    stats_fijo = construir_diccionario(train_df)    
    resultados = []
    for size in train_sizes:
        # Tomar una parte del conjunto de entrenamiento
        train_subset = train_df[:size]
        stats = construir_diccionario(train_subset)  # Diccionario con el mismo vocabulario base
        acc = evaluar_modelo(test_df, stats)
        resultados.append((size, acc))
        print(f"Tamaño Train: {size}, accuracy: {acc:.4f}")
    return resultados


# Main
if __name__ == "__main__":
    ruta_csv = "./CRI_Practica3/FinalStemmedSentimentAnalysisDataset.csv"
      # Trabajamos con un subconjunto pequeño

    df = cargar_dataset(ruta_csv, 100000)

    # Estrategia 1: Aumentar tamaño de entrenamiento
    estrategia_1(df, incrementos=[20000, 40000, 60000, 80000])

    # Estrategia 2: Modificar tamaño del diccionario
    estrategia_2(df, min_freqs=[1, 3, 5, 10])

    # Estrategia 3: Variar tamaño del conjunto de entrenamiento
    estrategia_3(df, train_sizes=[20000, 40000, 60000, 80000])



Estrategia 1: Aumentar tamaño de entrenamiento (diccionario variable)
Tamaño Train: 20000, accuracy: 0.7522
Tamaño Train: 40000, accuracy: 0.7508
Tamaño Train: 60000, accuracy: 0.7614
Tamaño Train: 80000, accuracy: 0.7616

Estrategia 2: Modificar tamaño del diccionario
Min. Frecuencia: 1, accuracy: 0.7597
Min. Frecuencia: 3, accuracy: 0.7514
Min. Frecuencia: 5, accuracy: 0.7462
Min. Frecuencia: 10, accuracy: 0.7386

Estrategia 3: Modificar tamaño del conjunto de entrenamiento (diccionario fijo)
Tamaño Train: 20000, accuracy: 0.7445
Tamaño Train: 40000, accuracy: 0.7512
Tamaño Train: 60000, accuracy: 0.7554
Tamaño Train: 80000, accuracy: 0.7597


# PART A